# Surface forcing at Southern Ocean Flux Station

This notebook prepares input data for GOTM in a Southern Ocean Flux Station case using LES data from [Large et al., 2019](https://doi.org/10.1175/JPO-D-18-0066.1).

In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd
from ruamel.yaml import YAML
sys.path.append(os.path.join(os.pardir, 'gotmtool'))
from gotmtool import *

In [ ]:
gotmtool_config = os.path.join(os.pardir, 'gotmtool', '.gotm_env.yaml')
gotm_env = config_load(gotmtool_config)

In [ ]:
casename = 'AprS'
inputdir = os.path.join(gotm_env['gotmdir_data'], 'SOFS', 'LES')
if casename == 'AprS':
    starttime = '2010-04-29 00:00:00'
    stoptime   = '2010-05-01 04:30:00'
    inputfile = 'les-april_2010.with_stokes.nc'
elif casename == "AprN":
    starttime = "2010-04-29 00:00:00"
    stoptime  = "2010-05-01 02:00:00"
    inputfile="les-april_2010.pp.no_stokes.nc"
elif casename== "JunS":
    starttime = "2010-06-07 06:45:00"
    stoptime  = "2010-06-08 23:45:00"
    inputfile = "les-june_2010.pp.with_stokes.nc"
elif casename == "JunN":
    starttime = "2010-06-07 06:45:00"
    stoptime  = "2010-06-08 21:15:00"
    inputfile = "les-june_2010.pp.no_stokes.nc"
else:
    raise ValueError('Case {:s} not supported. Stop.'.format(casename))
outputdir = 'data_{:s}'.format(casename)
os.makedirs(outputdir, exist_ok=True)

File `data.yaml` contains the information of different variables for conversion. Loop over all vairables, process and save the data.

In [ ]:
yamlfile = 'data.yaml'
with open(yamlfile, 'r') as f:
    fstring = f.read()
variables = YAML().load(fstring)

In [ ]:
ds = xr.load_dataset(os.path.join(inputdir, inputfile))
dttime = pd.to_datetime(starttime)+pd.to_timedelta(ds.time, unit='s')
depth = -np.abs(ds.zu.values)*0.01 # cm -> m

if '_FillValue' in ds.attrs:
    skipvalue = ds.attrs['_FillValue']
elif 'missing_value' in ds.attrs:
    skipvalue = ds.attrs['missing_value']
else:
    skipvalue = None

for var in variables:
    branch = variables[var]
    lname  = branch['longname']
    print('{:s}: {:s}'.format(var, lname))
    vtype  = branch['vartype']
    if 'scale_factor' in branch:
        factor = branch['scale_factor']
    else:
        factor = 1.    
    oname  = os.path.join(outputdir, branch['outname'])
    if vtype == 'scalar':
        # scalar variable
        vname = branch['varname']
        data = ds.data_vars[vname].values
        if len(data.shape) == 1:
            dat_dump_ts(dttime, [data], oname, skip_value=skipvalue, scale_factor=factor)
        else:
            dat_dump_pfl(dttime, depth, [data], oname, skip_value=skipvalue, scale_factor=factor)
    elif vtype == 'vector':
        # vector variable
        vnamex = branch['varnamex']
        vnamey = branch['varnamey']
        datax = ds.data_vars[vnamex].values
        datay = ds.data_vars[vnamey].values
        if len(datax.shape) == 1:
            dat_dump_ts(dttime, [datax, datay], oname, skip_value=skipvalue, scale_factor=factor)
        else:
            dat_dump_pfl(dttime, depth, [datax, datay], oname, skip_value=skipvalue, scale_factor=factor)
   